## Deliverable 2. Create a Customer Travel Destinations Map.

In [44]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import ipywidgets as widgets

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [45]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Filipe,CV,14.8961,-24.4956,78.53,75,0,9.28,clear sky
1,1,Nanortalik,GL,60.1432,-45.2371,33.37,96,100,2.26,snow
2,2,Souillac,MU,-20.5167,57.5167,75.36,73,40,14.97,scattered clouds
3,3,Hasaki,JP,35.7333,140.8333,64.29,55,98,16.40,overcast clouds
4,4,Bluff,NZ,-46.6000,168.3333,53.13,65,6,16.31,clear sky


In [46]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
minimum_temp = float(input('What is the minimum temperature you would like for your trip? '))
maximum_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [47]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_df = city_data_df.loc[(city_data_df['Max Temp'] <= maximum_temp) & \
                                       (city_data_df['Max Temp'] >= minimum_temp)]
filtered_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Filipe,CV,14.8961,-24.4956,78.53,75,0,9.28,clear sky
2,2,Souillac,MU,-20.5167,57.5167,75.36,73,40,14.97,scattered clouds
7,7,Rikitea,PF,-23.1203,-134.9692,75.36,77,7,15.19,light rain
9,9,Kloulklubed,PW,7.0419,134.2556,88.30,73,34,11.39,scattered clouds
11,11,Palauig,PH,15.4350,119.9045,86.38,64,44,4.21,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
711,711,Mogadishu,SO,2.0371,45.3438,78.46,82,54,13.00,broken clouds
712,712,Ati,TD,13.2154,18.3353,83.95,40,60,7.85,broken clouds
716,716,Veraval,IN,20.9000,70.3667,79.99,69,75,14.09,broken clouds
717,717,Sisophon,KH,13.5859,102.9737,75.96,82,100,5.97,overcast clouds


In [48]:
# 4a. Determine if there are any empty rows.
filtered_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [49]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_df = filtered_df.dropna()
filtered_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Filipe,CV,14.8961,-24.4956,78.53,75,0,9.28,clear sky
2,2,Souillac,MU,-20.5167,57.5167,75.36,73,40,14.97,scattered clouds
7,7,Rikitea,PF,-23.1203,-134.9692,75.36,77,7,15.19,light rain
9,9,Kloulklubed,PW,7.0419,134.2556,88.30,73,34,11.39,scattered clouds
11,11,Palauig,PH,15.4350,119.9045,86.38,64,44,4.21,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
711,711,Mogadishu,SO,2.0371,45.3438,78.46,82,54,13.00,broken clouds
712,712,Ati,TD,13.2154,18.3353,83.95,40,60,7.85,broken clouds
716,716,Veraval,IN,20.9000,70.3667,79.99,69,75,14.09,broken clouds
717,717,Sisophon,KH,13.5859,102.9737,75.96,82,100,5.97,overcast clouds


In [50]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sao Filipe,CV,78.53,clear sky,14.8961,-24.4956,
2,Souillac,MU,75.36,scattered clouds,-20.5167,57.5167,
7,Rikitea,PF,75.36,light rain,-23.1203,-134.9692,
9,Kloulklubed,PW,88.30,scattered clouds,7.0419,134.2556,
11,Palauig,PH,86.38,scattered clouds,15.4350,119.9045,
12,Dubbo,AU,75.33,clear sky,-32.2500,148.6167,
15,Carutapera,BR,77.74,broken clouds,-1.1950,-46.0200,
19,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,
26,Cabo San Lucas,MX,81.70,clear sky,22.8909,-109.9124,
29,Cayenne,GF,82.09,scattered clouds,4.9333,-52.3333,


In [51]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [52]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sao Filipe,CV,78.53,clear sky,14.8961,-24.4956,Hotel Xaguate
2,Souillac,MU,75.36,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
7,Rikitea,PF,75.36,light rain,-23.1203,-134.9692,People ThankYou
9,Kloulklubed,PW,88.30,scattered clouds,7.0419,134.2556,Storyboard Beach Resort
11,Palauig,PH,86.38,scattered clouds,15.4350,119.9045,Alperi Botique Resort
...,...,...,...,...,...,...,...
711,Mogadishu,SO,78.46,broken clouds,2.0371,45.3438,Hotel Juba
712,Ati,TD,83.95,broken clouds,13.2154,18.3353,
716,Veraval,IN,79.99,broken clouds,20.9000,70.3667,Lords Inn Somnath
717,Sisophon,KH,75.96,overcast clouds,13.5859,102.9737,Pyramid Hotel


In [53]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [54]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))